# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
!pip instal gmaps
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv("../output_data/cities.csv")
cities_df.head()

,City,Country,Lat,Lng,Cloudiness,Humidity,Max Temp,Wind Speed
0,Tuatapere,NZ,-46.13,167.68,18,75,55.00,4.00
1,Belmonte,BR,-15.86,-38.88,40,78,80.60,9.17
2,Kalakamati,BW,-20.59,27.31,75,75,64.42,10.22
3,Montepuez,MZ,-13.13,39.00,73,98,67.86,2.80
4,Avarua,CK,-21.21,-159.78,100,88,80.60,18.34


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

coordinates = []
for index, rows in cities_df.iterrows(): 
    coord_list = [rows["Lat"], rows["Lng"]] 
    coordinates.append(coord_list) 

fig = gmaps.figure()
heatmap = gmaps.heatmap_layer(coordinates, weights = cities_df["Humidity"])
heatmap.max_intensity = 100
fig.add_layer(heatmap)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
hotel_df = cities_df.loc[(cities_df['Max Temp'] > 75) & (cities_df['Max Temp'] < 80)
                     & (cities_df['Wind Speed'] < 10) & (cities_df['Cloudiness'] == 0)]

hotel_df

,City,Country,Lat,Lng,Cloudiness,Humidity,Max Temp,Wind Speed
96,Mildura,AU,-34.20,142.15,0,43,79.00,8.05
319,Oussouye,SN,12.48,-16.55,0,83,75.20,9.17
430,Sawākin,SD,19.11,37.33,0,65,75.76,8.19
434,Cruzeiro do Sul,BR,-29.51,-51.99,0,78,77.00,1.99
439,San Patricio,MX,19.22,-104.70,0,75,77.79,7.87


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df.loc[:,"Hotel Name"] = ''

base_url = "https://maps.googleapis.com/maps/api/place/textsearch/json"

target_radius = 5000
params = {
    "location": "",  
    "type": "lodging",
    "radius" : target_radius,
    "query": "hotel",
    "key": g_key,
}

for index, row in hotel_df.iterrows():

    params['location'] = f"{row['Lat']},{row['Lng']}"

    print(f"Finding hotels in {row['City']} within 5000 meters. ")
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("No hotel withing 5000 meters. Skipping.")
print("Search complete.")

/Users/neilhsu/anaconda3/envs/PythonData/lib/python3.6/site-packages/pandas/core/indexing.py:844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/neilhsu/anaconda3/envs/PythonData/lib/python3.6/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Finding hotels in Mildura within 5000 meters. 
Closest hotel is Quality Hotel Mildura Grand.
Finding hotels in Oussouye within 5000 meters. 
Closest hotel is Hôtel Katakalousse.
Finding hotels in Sawākin within 5000 meters. 
Closest hotel is فندق دشه سواكن.
Finding hotels in Cruzeiro do Sul within 5000 meters. 
Closest hotel is Zallon Hotel Executivo.
Finding hotels in San Patricio within 5000 meters. 
Closest hotel is Grand Isla Navidad Resort.
Search complete.


In [6]:
hotel_df.head()

,City,Country,Lat,Lng,Cloudiness,Humidity,Max Temp,Wind Speed,Hotel Name
96,Mildura,AU,-34.20,142.15,0,43,79.00,8.05,Quality Hotel Mildura Grand
319,Oussouye,SN,12.48,-16.55,0,83,75.20,9.17,Hôtel Katakalousse
430,Sawākin,SD,19.11,37.33,0,65,75.76,8.19,فندق دشه سواكن
434,Cruzeiro do Sul,BR,-29.51,-51.99,0,78,77.00,1.99,Zallon Hotel Executivo
439,San Patricio,MX,19.22,-104.70,0,75,77.79,7.87,Grand Isla Navidad Resort


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [8]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))